In [16]:
from load_data import load_expert_data
from utils_analysis import sort_by_key
import pandas as pd

def get_overview_table(expert_data):
    row_dicts = []
    data_by_triple = sort_by_key(expert_data, ['relation', 'concept', 'property'])
    #all_workers = data_by_worker.keys()
    #all_workers = set([d['workerid'] for d in expert_data])
    workers_exclude = set(['pia_test1'])
    for t, data in data_by_triple.items():
        triple_dict = dict()
        triple_dict['triple'] = t
        for d in data:
            w = d['workerid']
            if 'answer' in d and w not in workers_exclude:
                a = d['answer']
                expected_disagreements = []
                for k, v in d.items():
                    if k.startswith('disagreement_') and v == True:
                        expected_disagreements.append(k)
                triple_dict[f'answer-{w}'] = a
                triple_dict[f'expected_behavior-{w}'] = '-'.join(sorted(expected_disagreements))
                if 'reason' in d:
                    triple_dict[f'reason-{w}'] = d['reason']
                if 'comment' in d:
                    triple_dict[f'comment-{w}'] = d['comment']
        row_dicts.append(triple_dict)
    return row_dicts

run = 4
n_q = '*'
batch = '1'
group = 'reason_agreement_expert_inspection1'
expert_data = load_expert_data(run, group, n_q, batch)
expert_rows = get_overview_table(expert_data)

overview_df = pd.DataFrame(expert_rows)
overview_df.to_csv(f'../analyses/expert_annotations/{group}-overview.csv')




run4-group_reason_agreement_expert_inspection1/qu30-s_qu30-batch1.csv
no summary data


In [17]:
overview_df

,answer-antske,answer-pia run1,answer-piek,comment-antske,comment-pia run1,expected_behavior-antske,expected_behavior-pia run1,expected_behavior-piek,reason-antske,reason-pia run1,triple
0,False,False,NaN,NaN,Princeton Wordnet uses the following phrase as...,disagreement_agreement,disagreement_agreement,NaN,NaN,Differences in what is typical.,typical_of_property-arrow-fly
1,True,True,NaN,NaN,NaN,disagreement_agreement,disagreement_agreement,NaN,Whether something is considered typical or not...,NaN,typical_of_concept-arrow-fly
2,True,True,True,NaN,NaN,disagreement_agreement,disagreement_agreement,disagreement_agreement,"if anything, people thinking that no one uses ...",broad-vs narrow interpretation of flying,afforded_usual-arrow-fly
3,False,False,NaN,NaN,Possible category- arrows used for decoration,disagreement_agreement,disagreement_agreement-disagreement_conceptual...,NaN,NaN,,variability_limited-arrow-fly
4,False,False,NaN,NaN,NaN,disagreement_agreement,disagreement_agreement,NaN,NaN,NaN,creative-arrow-fly
5,False,False,NaN,,Some people might say that something which can...,disagreement_agreement,disagreement_agreement,NaN,NaN,NaN,rare-arrow-fly
6,False,False,NaN,NaN,NaN,disagreement_agreement,disagreement_agreement,NaN,NaN,NaN,unusual-arrow-fly
7,False,False,NaN,"Especially with the `can', I can imagine some ...",NaN,disagreement_agreement-disagreement_ambiguous_...,disagreement_agreement,NaN,NaN,NaN,impossible-arrow-fly
8,False,False,NaN,NaN,NaN,disagreement_agreement,disagreement_agreement,NaN,NaN,The creativity question probably has the bigge...,creative-pin-roll
9,False,False,NaN,NaN,NaN,disagreement_agreement,disagreement_agreement,NaN,,NaN,unusual-pin-roll
